In [13]:
from sklearn.datasets import load_iris
import numpy as np

# Aim: Perform Batch Gradient Descent with Early stopping for Softmax Regression without scikit-learn

### 1. Data Loading

In [4]:
iris = load_iris(as_frame=True) # load data as a dataframe

list(iris)

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [75]:
X = iris["data"].values   # get data as numpy array

y = iris["target"].values # get target label as numpy array 

X_with_bias = np.c_[np.ones(len(X)), X] # (or) easier method - add_dummy_feature(X)

n_inp_features = X_train.shape[1] # 3 featues and 1 bias term - input
n_out_classes  = len(np.unique(y)) # 3 output classes

### 2. Test train split

In [39]:
test_ratio = 0.2                           # test ratio to total size
total_size = len(X_with_bias)

test_size = int(test_ratio * total_size)  # number of training data
train_size = total_size - test_size       # number of testing data 

np.random.seed(42)
rdn_idx = np.random.permutation(total_size) # shuffle from 0 to 149

X_train = X_with_bias[rdn_idx[:train_size]] # training data
X_test = X_with_bias[rdn_idx[train_size:]] # test data 
y_train = y[rdn_idx[:train_size]]          # training data label
y_test = y[rdn_idx[train_size:]]           # testing data label

### 3. Encoding
Since softmax regreesion is to be done on target class probabilities that is the values need to be 0 or 1, the target indices needs to be encoded. Therefore, one hot encoding needs to be done

In [104]:
def one_hot(y):
    n_cats = np.ones(y.max()+1) # number of categories    
    cat_diag = np.diag(n_cats) # diagonalise the categories
    
    return cat_diag[y]        # encoding

In [105]:
y_train_enc = one_hot(y_train)
y_test_enc = one_hot(y_test)

### 4. Scaling
Scaling neeeds to be done for the GD to work rpoperly and give a values without any bias. Perform `Z-transform`, that is
$$z = \frac{x - \mu}{\sigma}$$

In [68]:
mean    = X_train[:, 1:].mean(axis=0) # scale all features except bias term
std_dev = X_train[:, 1:].std(axis=0)

X_train[:, 1:] = (X_train[:,1:] - mean)/std_dev # z formula

mean    = X_test[:, 1:].mean(axis=0) # scale all features except bias term
std_dev = X_test[:, 1:].std(axis=0)

X_test[:, 1:] = (X_test[:,1:] - mean)/std_dev

### 5. Training
Softmax function is of the form,
$$p_i = S(z_i) = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_i}}$$, p - vector of probability of each class

The loss function for softmax is of the form,
$$J(\mathbf{\Theta}) = -\frac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$$

The gradient of the loss function is of the form,
$$\nabla_\theta J(\mathbf{\Theta}) = \frac{1}{m}\sum\limits_{i=1}^{m}(\hat{p}_k^{(i)} - y_k^{(i)})x^{i}$$

In [98]:
def softmax(logits):
    numer = np.exp(logits) # exponential of each term

    denom = numer.sum(axis=1, keepdims=True) # keepdims - required for broadcasting

    return numer / denom

In [117]:
lr = 0.1                    # learning rate
epochs = 1500               # number of epochs
m = train_size              # to compute the average
epsilon = 1e-5

np.random.seed(42)
theta = np.random.randn(n_inp_features, n_out_classes) # 5X3 random initial parameter value

for epoch in range(epochs):
    logits = X_train @ theta # logits - output of normal linear regression

    y_prob = softmax(logits) # softmax of y

    if epoch % 100 == 0 or epochs <= 10:
        y_prob_test = softmax(X_test @ theta)

        test_cross_entropy_loss = -(y_test_enc * np.log(y_prob_test + epsilon)).mean() # epsilon - to prevent from getting NaN if pprobability is 0

        print(f"At epoch : {epoch} | loss : {test_cross_entropy_loss}")

    err = y_prob - y_train_enc    # error - offset away from actual

    grads = 1/m * X_train.T @ err # gradient

    theta -= lr * grads           # step

At epoch : 0 | loss : 1.2910169335356907
At epoch : 100 | loss : 0.14209276578243565
At epoch : 200 | loss : 0.11259462605070966
At epoch : 300 | loss : 0.0968287305479978
At epoch : 400 | loss : 0.08696449280252114
At epoch : 500 | loss : 0.08032410664891683
At epoch : 600 | loss : 0.07559923941956628
At epoch : 700 | loss : 0.07208461235987478
At epoch : 800 | loss : 0.06937447461566741
At epoch : 900 | loss : 0.0672223505399955
At epoch : 1000 | loss : 0.0654713951182708
At epoch : 1100 | loss : 0.06401775070213839
At epoch : 1200 | loss : 0.06279025581422021
At epoch : 1300 | loss : 0.06173868967763608
At epoch : 1400 | loss : 0.06082668579165255


In [118]:
theta # fitted model parameters

array([[ 0.10123407,  2.75412715, -1.84922283],
       [-1.11124607,  1.03434243,  1.13164316],
       [ 2.43081281,  0.02427266, -0.57791232],
       [-2.62725075, -0.0571928 ,  2.29785615],
       [-2.92640378, -2.36350271,  1.89367068]])

### 6. Prediction

In [128]:
X_pred = X_test[5].reshape(-1,1)
y_hat = y_test[5]

y_prob = X_pred.T @ theta # prediccted probability of each class

y_pred = np.argmax(y_prob)
y_pred, y_hat

(np.int64(1), np.int64(1))

### 7. Accuracy Score
Lets test on test data

In [169]:
y_prob = X_test @ theta 
y_pred = np.argmax(y_prob, axis=1)

accuracy = (y_pred == y_test).sum()/len(y_test) * 100 # accuracy formula - or (y_pred == y_test).mean()
print(f"Accuracy is {accuracy.item()}%")

Accuracy is 90.0%


# Additional

### a.if regularisation is done (L2 is perfomred)

In [197]:
lr = 0.1                    # learning rate
epochs = 1500               # number of epochs
m = train_size              # to compute the average
epsilon = 1e-5
alpha = 0.01                # regularisation hyperparameter

np.random.seed(42)
theta = np.random.randn(n_inp_features, n_out_classes) # 5X3 random initial parameter value

for epoch in range(epochs):
    logits = X_train @ theta # logits - output of normal linear regression

    y_prob = softmax(logits) # softmax of y

    if epoch % 100 == 0 or epochs <= 10:
        y_prob_test = softmax(X_test @ theta)

        test_cross_entropy_loss = -(y_test_enc * np.log(y_prob_test + epsilon)) # epsilon - to prevent from getting NaN if pprobability is 0

        l2_loss = 1/2 * (theta[1:]**2).sum() # to be done on all featues except bias

        total_loss = test_cross_entropy_loss.sum(axis=1).mean() + alpha * l2_loss

        print(f"At epoch : {epoch} | loss : {total_loss}")

    err = y_prob - y_train_enc    # error - offset away from actual

    grads = 1/m * X_train.T @ err # gradient

    grads += np.r_[np.zeros([1,n_out_classes]), alpha/m * theta[1:]] # concatenate along axis0 to add reg value to all params

    theta -= lr * grads           # step

At epoch : 0 | loss : 3.9388163174414963
At epoch : 100 | loss : 0.48123501892987247
At epoch : 200 | loss : 0.4062102320253216
At epoch : 300 | loss : 0.3712593362241967
At epoch : 400 | loss : 0.35361467575107003
At epoch : 500 | loss : 0.34524670346484576
At epoch : 600 | loss : 0.3421836211532245
At epoch : 700 | loss : 0.34229269753743674
At epoch : 800 | loss : 0.3443648743170089
At epoch : 900 | loss : 0.34768247627464144
At epoch : 1000 | loss : 0.35180249649327805
At epoch : 1100 | loss : 0.3564423385762572
At epoch : 1200 | loss : 0.3614167437752964
At epoch : 1300 | loss : 0.36660150829610083
At epoch : 1400 | loss : 0.37191183641452463


In [198]:
y_prob = X_test @ theta 
y_pred = np.argmax(y_prob, axis=1)

accuracy = (y_pred == y_test).sum()/len(y_test) * 100 # accuracy formula - or (y_pred == y_test).mean()
print(f"Accuracy is {accuracy.item()}%")

Accuracy is 90.0%


### b. Early stopping
Tuning the hyperparameter alpha doesn;t seem to vary the accuracy, therefore lets try tuning the hyperparameter

In [254]:
lr = 0.1                    # learning rate
epochs = 1500               # number of epochs
m = train_size              # to compute the average
epsilon = 1e-5
alpha = 0.01                # regularisation hyperparameter

best_loss = np.inf

np.random.seed(42)
theta = np.random.randn(n_inp_features, n_out_classes) # 5X3 random initial parameter value

for epoch in range(epochs):
    logits = X_train @ theta # logits - output of normal linear regression

    y_prob = softmax(logits) # softmax of y

    y_prob_test = softmax(X_test @ theta)

    test_cross_entropy_loss = -(y_test_enc * np.log(y_prob_test + epsilon)) # epsilon - to prevent from getting NaN if pprobability is 0

    l2_loss = 1/2 * (theta[1:]**2).sum() # to be done on all featues except bias

    total_loss = test_cross_entropy_loss.sum(axis=1).mean() + alpha * l2_loss

    if epoch % 100 == 0 or epochs <= 10:
        print(f"At epoch : {epoch} | loss : {total_loss}")
    
    if total_loss < best_loss: # condition for early stopping
        best_loss = total_loss

    else:
        print(epoch-1, np.round(best_loss,4), "early stopping!")
        break

    err = y_prob - y_train_enc    # error - offset away from actual

    grads = 1/m * X_train.T @ err # gradient

    grads += np.r_[np.zeros([1,n_out_classes]), alpha/m * theta[1:]] # concatenate along axis0 to add reg value to all params

    theta -= lr * grads           # step

At epoch : 0 | loss : 3.9388163174414963
At epoch : 100 | loss : 0.48123501892987247
At epoch : 200 | loss : 0.4062102320253216
At epoch : 300 | loss : 0.3712593362241967
At epoch : 400 | loss : 0.35361467575107003
At epoch : 500 | loss : 0.34524670346484576
At epoch : 600 | loss : 0.3421836211532245
644 0.3419 early stopping!


In [255]:
y_prob = X_test @ theta 
y_pred = np.argmax(y_prob, axis=1)

accuracy = (y_pred == y_test).sum()/len(y_test) * 100 # accuracy formula - or (y_pred == y_test).mean()
print(f"Accuracy is {accuracy.item()}%")

Accuracy is 93.33333333333333%


We can see that there is an increase in performance by doing early stopping. Therefore by early stopping, overfitting of the model is prevented